In [ ]:
#Quick cell to make jupyter notebook use the full screen width
from IPython.core.display import display, HTML
display(HTML("<style>.container { width:100% !important; }</style>"))

In [ ]:
#Some plotting libraries
import matplotlib.pyplot as plt
%matplotlib notebook
from bokeh.plotting import show, save, output_notebook, output_file
from bokeh.layouts import gridplot
from bokeh.resources import INLINE 
output_notebook(resources=INLINE)

In [ ]:
from src import workflow
from src.data import Dataset

In [ ]:
import umap
import umap.plot
from sklearn.feature_extraction.text import CountVectorizer

In [ ]:
breweries_ds = Dataset.load("breweries_by_style")
breweries = breweries_ds.data

## Embed the data

We are going to vectorize our data and look at the number of categorical values they have in common.  A useful thing to do here is to require each row to have a minimum support before being included.  Filtering this early, will ensure indices line up later on.

This step turns a sequence of space seperated text into a sparse matrix of counts.  One row per row of our data frame and one column per unique token that appeared in our categorical field of interest.

If we want to deal with sets (i.e. just presence or absence of a category) use:<BR>
`beer_by_authors_vectorizer = CountVectorizer(binary=True)`<BR>
If we think counts should matter we might use:<BR>
`beer_by_authors_vectorizer = CountVectorizer()`<BR>
or if we want to correct for very unbalanced column frequencies:<BR>
`beer_by_authors_vectorizer = TfidfVectorizer()`<BR>
    
We probably want to remove any brewery that has only a few reviewers.  This threshold can be based on intuition or by examining the distribution of reviews per brewery.
    
We use `min_df=10` in our CountVectorize to only count styles that are made by at least 10 breweries.


In [ ]:
popular_breweries = breweries[breweries.num_reviewers>100].reset_index(drop=True)

In [ ]:
brewery_by_style_vectorizer = CountVectorizer(binary=False, min_df=10, token_pattern='[^,]+')
brewery_by_style = brewery_by_style_vectorizer.fit_transform(popular_breweries.beer_style)
brewery_by_style

Now we reduce the dimension of this data.

If we are dealing with sets (i.e. just presence or absence of a category) use:<BR>
`metric='jaccard'`<BR>
If we think counts should matter we might use:<BR>
`metric='hellinger'`<BR>
or if we want to correct for very unbalanced column frequencies:<BR>
`metric='hellinger'`<BR>
    
As you get more and more points I'd recommend increasing the `n_neighbors` parameter to compensate.  Thing of this as a resolution parameter.

`n_components` controls the dimension you will be embedding your data into (2-dimensions for easy visualization).  Feel free to embed into higher dimensions for clustering if you'd like.

`unique=True` says that if you have two identical points you want to map them to the exact same co-ordinates in your low space.  This becomes especially important if you have more exact dupes that your `n_neighbors` parameter.  That is the problem case where exact dupes can be pushed into very different regions of your space.

In [ ]:
%%time
brewery_by_style_model = umap.UMAP(n_neighbors=15, n_components=2, metric='hellinger', min_dist=0.1,
                                  unique=True, random_state=42).fit(brewery_by_style)

In [ ]:
outfile ='results/brewery_by_style_hellinger'

In [ ]:
umap_plot = umap.plot.points(brewery_by_style_model, labels=popular_breweries.favorite_style.values, theme='fire');
#umap_plot.figure.savefig(outfile+'.png', dpi=300, bbox_inches='tight')

... and now for an interactive plot with mouseover.

In [ ]:
hover_df = popular_breweries['brewery_name favorite_style beer_abv num_beer_style review_overall'.split()]
f = umap.plot.interactive(brewery_by_style_model, labels=popular_breweries.favorite_style.values, 
                          hover_data=hover_df, theme='fire', point_size=5);
#save(f,outfile+'.html')
show(f)

In [ ]:
seeds = [10, 200, 3000, 40000]
brewery_by_style_model_list = []

for seed in seeds:
    embed = umap.UMAP(n_neighbors=15, n_components=2, metric='hellinger', min_dist=0.1,
                                  unique=True, random_state=seed).fit(brewery_by_style)
    brewery_by_style_model_list.append(embed)

In [ ]:
hover_df = popular_breweries['brewery_name favorite_style beer_abv num_beer_style review_overall'.split()]
f = umap.plot.interactive(brewery_by_style_model_list[0], labels=popular_breweries.favorite_style.values, 
                          hover_data=hover_df, theme='fire', point_size=5);
#save(f,outfile+'.html')
show(f)

In [ ]:
hover_df = popular_breweries['brewery_name favorite_style beer_abv num_beer_style review_overall'.split()]
f = umap.plot.interactive(brewery_by_style_model_list[1], labels=popular_breweries.favorite_style.values, 
                          hover_data=hover_df, theme='fire', point_size=5);
#save(f,outfile+'.html')
show(f)

In [ ]:
hover_df = popular_breweries['brewery_name favorite_style beer_abv num_beer_style review_overall'.split()]
f = umap.plot.interactive(brewery_by_style_model_list[2], labels=popular_breweries.favorite_style.values, 
                          hover_data=hover_df, theme='fire', point_size=5);
#save(f,outfile+'.html')
show(f)

In [ ]:
hover_df = popular_breweries['brewery_name favorite_style beer_abv num_beer_style review_overall'.split()]
f = umap.plot.interactive(brewery_by_style_model_list[3], labels=popular_breweries.favorite_style.values, 
                          hover_data=hover_df, theme='fire', point_size=5);
#save(f,outfile+'.html')
show(f)

In [ ]:
hover_df = popular_breweries['brewery_name favorite_style beer_abv num_beer_style review_overall'.split()]

f0 = umap.plot.interactive(brewery_by_style_model_list[0], labels=popular_breweries.favorite_style.values, 
                          hover_data=hover_df, theme='fire', point_size=3)
f1 = umap.plot.interactive(brewery_by_style_model_list[1], labels=popular_breweries.favorite_style.values, 
                          hover_data=hover_df, theme='fire', point_size=3)
f2 = umap.plot.interactive(brewery_by_style_model_list[2], labels=popular_breweries.favorite_style.values, 
                          hover_data=hover_df, theme='fire', point_size=3)
f3 = umap.plot.interactive(brewery_by_style_model_list[3], labels=popular_breweries.favorite_style.values, 
                          hover_data=hover_df, theme='fire', point_size=3)
f = gridplot([[f0, f1], [f2, f3]], plot_width=500, plot_height=500)
show(f)